In [ ]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
random.seed(108)

Training images set

In [ ]:
!wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB-Training_fixed.zip
!unzip /content/GTSRB-Training_fixed.zip

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/major_project/kaggle/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download <name-of-competition>


In [ ]:
!curl -L "https://public.roboflow.com/ds/ONxyYx8ddR?key=36n4EfSQY5" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def cleanFolder(folderName):
  annotations = [os.path.join(folderName, x) for x in os.listdir(folderName)]
  for ann in tqdm(annotations):
    os.remove(ann)

In [ ]:
cleanFolder("/content/GTSRBedited/images");

100%|██████████| 74/74 [00:00<00:00, 22750.02it/s]


In [ ]:
!mkdir GTSRBedited
!mkdir GTSRBedited/images
!mkdir GTSRBedited/labels

In [ ]:
def saveLabels(csvPath, classId, fileNamePrefix):
  data  = pd.read_csv(csvPath, sep = ';')

  for i in range(len(data["Width"])):
    imgWidth = data["Width"][i]
    imgHeight = data["Height"][i]
    b_center_x = ((data["Roi.X1"][i] + data["Roi.X2"][i]) / 2)/imgWidth 
    b_center_y = ((data["Roi.Y1"][i] + data["Roi.Y2"][i]) / 2)/imgHeight
    b_width    = ((data["Roi.X2"][i] - data["Roi.X1"][i]))/imgWidth
    b_height   = ((data["Roi.Y2"][i] - data["Roi.Y1"][i]))/imgHeight

    correspondingImageName = data["Filename"][i];

    destFolder = "/content/GTSRBedited/labels/"

    finalPath  = destFolder + fileNamePrefix + correspondingImageName.replace('ppm', 'txt')

    f = open(finalPath, "w")
    f.write(str(classId)+" "+str(b_center_x)+" "+str(b_center_y)+" "+str(b_width)+" "+str(b_height))
    


In [ ]:
csvPath = ["/content/GTSRB/Training/00000/GT-00000.csv", "/content/GTSRB/Training/00007/GT-00007.csv", "/content/GTSRB/Training/00014/GT-00014.csv"]

In [ ]:
saveLabels(csvPath[0], 11, "00000_")
saveLabels(csvPath[1], 12, "00007_")
saveLabels(csvPath[2], 13, "00014_")


In [ ]:
def convertPpmToJpgAndSave(imagesFolder, prefix):
  imagesPath = [os.path.join(imagesFolder, x) for x in os.listdir(imagesFolder) if x[-3:] == "ppm"]
  imagesName = os.listdir(imagesFolder)
  
  #print(imagesName[0], imagesPath[0], imagesName[3], imagesPath[3])
  
  imagesPath.sort()
  imagesName.sort()

  #sort is necessary above, to find out check the loop below
  #or remove above two lines and exectue

  for i in range(len(imagesPath)):
    
    imageInPpm = Image.open(imagesPath[i])
    if "csv" in imagesName[i]:
      print(imagesName[i], " at index ", i, " length is ", len(imagesPath))
      break;
    destFolder = "/content/GTSRBedited/images/"
    PathOfImageInJpg = destFolder + prefix + imagesName[i].replace("ppm", "jpg")
    imageInPpm.save(PathOfImageInJpg)

In [ ]:
imagesPath = ["/content/GTSRB/Training/00000/", "/content/GTSRB/Training/00007/", "/content/GTSRB/Training/00014/"]

In [ ]:
convertPpmToJpgAndSave(imagesPath[0], "00000_")
convertPpmToJpgAndSave(imagesPath[1], "00007_")
convertPpmToJpgAndSave(imagesPath[2], "000014_")


In [ ]:
def moveFilesFrom(sourceFolder, toDestFolder):
  imagesInSourceFolder = os.listdir(sourceFolder);
  for imageName in tqdm(imagesInSourceFolder):
    sourceFilePath = sourceFolder + imageName
    destFilePath = toDestFolder + imageName
    shutil.move(sourceFilePath, destFilePath)


In [ ]:
sourceFolder = ["/content/GTSRBedited/images/", "/content/GTSRBedited/labels/"]
toDestFolder = ["/content/export/images/", "/content/export/labels/"]
for i in range(len(sourceFolder)):
  moveFilesFrom(sourceFolder[i], toDestFolder[i])

100%|██████████| 1650/1650 [00:00<00:00, 33085.86it/s]


In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
# Function to get the data from XML Annotation
def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()
    
    # Initialise the info dict 
    info_dict = {}
    info_dict['bboxes'] = []

    # Parse the XML Tree
    for elem in root:
        # Get the file name 
        if elem.tag == "filename":
            info_dict['filename'] = elem.text
            
        # Get the image size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))
            
            info_dict['image_size'] = tuple(image_size)
        
        # Get details of the bounding box 
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                    
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)            
            info_dict['bboxes'].append(bbox)
    
    return info_dict

In [ ]:
print(extract_info_from_xml('annotations/road4.xml'))


In [ ]:
# Dictionary that maps class names to IDs
class_name_to_id_mapping = {"trafficlight": 0,
                           "stop": 1,
                           "speedlimit": 2,
                           "crosswalk": 3}

# Convert the info dict to the required yolo format and write it to disk
def convert_to_yolov5(info_dict):
    print_buffer = []
    
    # For each bounding box
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        
        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h, image_c = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        #Write the bbox details to the file 
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    # Name of the file which we have to save 
    save_file_name = os.path.join("annotations", info_dict["filename"].replace("png", "txt"))
    
    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [ ]:
# Read images and annotations

trainImagesPath = "/content/export/images/"
labelsPath = "/content/export/labels/"

images = [os.path.join(trainImagesPath, x) for x in os.listdir(trainImagesPath)]
annotations = [os.path.join(labelsPath, x) for x in os.listdir(labelsPath)]

images.sort()
annotations.sort()

# Split the dataset into train-valid-test splits 
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
#val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

In [ ]:
!mkdir train
!mkdir train/images train/labels
!mkdir train/images/train train/images/val train/labels/train train/labels/val

In [ ]:
#Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
move_files_to_folder(train_images, 'train/images/train')
move_files_to_folder(val_images, 'train/images/val/')
#move_files_to_folder(test_images, 'images/test/')
move_files_to_folder(train_annotations, 'train/labels/train/')
move_files_to_folder(val_annotations, 'train/labels/val/')
#move_files_to_folder(test_annotations, 'annotations/test/')

In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 14992, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 14992 (delta 0), reused 1 (delta 0), pack-reused 14989
Receiving objects: 100% (14992/14992), 14.01 MiB | 16.50 MiB/s, done.
Resolving deltas: 100% (10294/10294), done.


In [ ]:
!python /content/yolov5/train.py --img 480 --cfg /content/yolov5/models/yolov5n.yaml --hyp /content/yolov5/data/hyps/hyp.scratch-low.yaml --batch 128 --epochs 25 --cache ram --weights yolov5s.pt --data /content/yolov5/data/data.yaml  --workers 24 --name udacity_gtsrb

train: weights=yolov5s.pt, cfg=/content/yolov5/models/yolov5n.yaml, data=/content/yolov5/data/data.yaml, hyp=/content/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=25, batch_size=128, imgsz=480, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=24, project=yolov5/runs/train, name=udacity_gtsrb, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-70-g589edc7 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0,

CHANGE the "source" and "weights" attribute below
Take weights as best.pt from path mentioned by yolo trainer after training is finished ---> "Results saved to yolov5/runs ..... "

In [ ]:
!python yolov5/detect.py --source https://www.gannett-cdn.com/presto/2018/09/09/PVCS/4e811a87-1147-4195-b237-90d27238a8fa-20180909_220853000_iOS.jpg  --weights /content/drive/MyDrive/major_project/best.pt




detect: weights=['/content/drive/MyDrive/major_project/best.pt'], source=https://www.gannett-cdn.com/presto/2018/09/09/PVCS/4e811a87-1147-4195-b237-90d27238a8fa-20180909_220853000_iOS.jpg, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
Found https://www.gannett-cdn.com/presto/2018/09/09/PVCS/4e811a87-1147-4195-b237-90d27238a8fa-20180909_220853000_iOS.jpg locally at 4e811a87-1147-4195-b237-90d27238a8fa-20180909_220853000_iOS.jpg
YOLOv5 🚀 v7.0-69-g3b6e27a Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7020913 parameters, 0 gradients
image 1/1 /content/4e811a87-1147-4195-b

In [ ]:
print(result.length)

In [ ]:
isStop = "stop car" if ("stop" in result) else 0
isSpeedLimit = "slow down the car" if ("speedlimit" in result) else 0
isTrafficLight = "traffic light detected" if("traffic" in result) else 0
print(isStop, "...", isSpeedLimit, "...", isTrafficLight, "...")

0 ... 0 ... 0 ...
